In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import os
from datetime import datetime

from helper_functions import *

In [2]:
# Loading the lookup table
df_lookup = pd.read_csv('../input/store_to_pool_lookup.csv', dtype=str)
df_lookup.head()

,Store#,Pool_ID,Pool#,AddressName
0,0001,FRA,2693,Bed Bath and Beyond #2693\nc/o Ryder Integrate...
1,0006,SPR,2690,Palmer Dedicated Logistics
2,0008,SCA,2613,Bed Bath and Beyond #2613 c/o Taylored Services
3,0009,FRA,2693,Bed Bath and Beyond #2693\nc/o Ryder Integrate...
4,0012,SPR,2690,Palmer Dedicated Logistics


In [3]:
# Getting the current proejct directory
project_dir = os.path.dirname(os.getcwd())
project_dir = project_dir.replace('\\','/')
selected_folder = 'sample'

files = [file for file in os.listdir('../input/'+ selected_folder + '/')]
inputFolderPath = project_dir + "/input/" + selected_folder + "/"
log_df = pd.DataFrame(columns=['PO#/Ext. Doc#', 'BC SO#', 'SSH#', 'Prev. Store/Pool#', 'Prev. Store/Pool Name', 'New Pool#', 'New Pool Name'])

for file in progressbar(files, "Progress: ", 60):
    inputFilePath = inputFolderPath + file
    tree = ET.parse(inputFilePath)
    root = tree.getroot()

    # Finding the reference ID
    shipment = root.find('{http://www.spscommerce.com/RSX}Shipment')
    header = shipment.find('{http://www.spscommerce.com/RSX}Header')
    address = header.find('{http://www.spscommerce.com/RSX}Address')
    addrLocation = address.find('{http://www.spscommerce.com/RSX}AddressLocationNumber')
    addrName = address.find('{http://www.spscommerce.com/RSX}AddressName')

    # Logging document details
    reference = header.find('{http://www.spscommerce.com/RSX}Reference')
    reference_ID = reference.find('{http://www.spscommerce.com/RSX}ReferenceID')  # SSH Number
    order_level = shipment.find('{http://www.spscommerce.com/RSX}OrderLevel')
    order_header = order_level.find('{http://www.spscommerce.com/RSX}OrderHeader')
    PONum = order_header.find('{http://www.spscommerce.com/RSX}PurchaseOrderNumber')
    SONum = order_header.find('{http://www.spscommerce.com/RSX}InternalOrderNumber')

    # Finding the Address Loaction  Number from the lookup table
    matchFound = "No"
    for i in df_lookup.index:
        if str(df_lookup.loc[i,"Store#"]) == addrLocation.text:
            matchFound = "Yes"
            newAddrLocation = str(df_lookup.loc[i,"Pool#"])
            newAddrName = str(df_lookup.loc[i,"AddressName"])
    
    if matchFound == "No":
        print(addrLocation.text + " : Match not found")
        print(addrLocation.text)

    # logging the change
    log_line = [PONum.text, SONum.text, reference_ID.text, addrLocation.text, addrName.text, newAddrLocation, newAddrName]
    log_df.loc[len(log_df.index)] = log_line

    # Updating the Address Location and Name
    addrLocation.text = newAddrLocation
    addrName.text = newAddrName
    
    # Exporting the modified file
    outputFolderPath = project_dir + "/output/"
    outputFilePath = outputFolderPath + file
    tree.write(outputFilePath, 'utf-8', True, 'http://www.spscommerce.com/RSX')

# Exporting the log file
timestamp = datetime.today()
timestamp = timestamp.strftime("%Y-%m-%d_%H-%M-%S")
log_filename = 'pool_update_log_' + timestamp + '.csv'
outputFilePath = outputFolderPath + log_filename
log_df.to_csv(outputFilePath, index=False)

Progress: [############################################################] 1/1

